# Part 2: Train, Check Bias, Record Lineage, and Register a Model

<a id='aud-overview'> </a>

## [Overview](./0-AutoClaimFraudDetection.ipynb)
* [Notebook 0 : Overview, Architecture and Data Exploration](./0-AutoClaimFraudDetection.ipynb)
* [Notebook 1: Data Prep, Process, Store Features](./1-data-prep-e2e.ipynb)
* **[Notebook 2: Train, Check Bias, Tune, Record Lineage, and Register a Model](./2-lineage-train-assess-bias-tune-registry-e2e.ipynb)**
  * **[Architecture](#train)**
  * **[Train a model using XGBoost](#aud-train-model)**
  * **[Model lineage with artifacts and associations](#model-lineage)**
  * **[Evaluate the model for bias with Clarify](#check-bias)**
  * **[Deposit Model and Lineage in SageMaker Model Registry](#model-registry)**
* [Notebook 3: Mitigate Bias, Train New Model, Store in Registry](./3-mitigate-bias-train-model2-registry-e2e.ipynb)
* [Notebook 4: Deploy Model, Run Predictions](./4-deploy-run-inference-e2e.ipynb)
* [Notebook 5 : Create and Run an End-to-End Pipeline to Deploy the Model](./5-pipeline-e2e.ipynb)

In this section we will show how you can assess pre-training and post-training bias with SageMaker Clarify, Train the Model using XGBoost on SageMaker, and then finally deposit it in the Model Registry, along with the Lineage of Artifacts that were created along the way: data, code and model metadata.

In this second model, you will fix the gender imbalance in the dataset using SMOTE and train another model using XGBoost. This model will also be saved to our registry and eventually approved for deployment.

<a id ='train'> </a>

## Architecture for the ML Lifecycle Stage: Train, Check Bias, Tune, Record Lineage, Register Model
[overview](#overview)
----

![train-assess-tune-register](./images/part2.png)

To apply the update to the current kernel, run the following code to refresh the kernel.

In [1]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Load stored variables
Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything you may need to create them again or it may be your first time running this notebook.

In [1]:
%store -r
%store

Stored variables and their in-db values:
bucket                             -> 'sagemaker-us-east-1-875692608981'
claims_fg_name                     -> 'fraud-detect-demo-claims'
claims_preprocessed                ->       policy_id  incident_severity  num_vehicles_i
claims_table                       -> 'fraud-detect-demo-claims-1637200704'
col_order                          -> ['fraud', 'customer_gender_male', 'customer_educat
customers_fg_name                  -> 'fraud-detect-demo-customers'
customers_preprocessed             ->       policy_id  customer_age  customer_education 
customers_table                    -> 'fraud-detect-demo-customers-1637200706'
database_name                      -> 'sagemaker_featurestore'
dataset_uri                        -> 's3://sagemaker-us-east-1-875692608981/fraud-detec
prefix                             -> 'fraud-detect-demo'
test_data_uri                      -> 's3://sagemaker-us-east-1-875692608981/fraud-detec
train                           

**<font color='red'>Important</font>: You must have run the previous sequential notebooks to retrieve variables using the StoreMagic command.**

### Import libraries

In [2]:
import json
import time
import boto3
import sagemaker
import numpy as np
import pandas as pd
import awswrangler as wr

from sagemaker.xgboost.estimator import XGBoost
from model_package_src.inference_specification import InferenceSpecification

### Set region, boto3 and SageMaker SDK variables

In [3]:
# You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

Using AWS Region: us-east-1


In [4]:
boto_session = boto3.Session(region_name=region)
s3_client = boto3.client("s3", region_name=region)
sagemaker_client = boto_session.client("sagemaker")
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client
)
sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client("sts").get_caller_identity()["Account"]

In [5]:
# variables used for parameterizing the notebook run
estimator_output_path = f"s3://{bucket}/{prefix}/training_jobs"
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

bias_report_1_output_path = f"s3://{bucket}/{prefix}/clarify-output/bias_1"

xgb_model_name = "xgb-insurance-claims-fraud-model"
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
predictor_instance_count = 1
predictor_instance_type = "ml.c5.xlarge"
batch_transform_instance_count = 1
batch_transform_instance_type = "ml.c5.xlarge"
claify_instance_count = 1
clairfy_instance_type = "ml.c5.xlarge"

<a id='aud-train-model'></a>

## Train a model using XGBoost

[overview](#overview)
----
Once the training and test datasets have been persisted in S3, you can start training a model by defining which SageMaker Estimator you'd like to use. For this guide, you will use the [XGBoost Open Source Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html) to train your model. This estimator is accessed via the SageMaker SDK, but mirrors the open source version of the [XGBoost Python package](https://xgboost.readthedocs.io/en/latest/python/index.html). Any functioanlity provided by the XGBoost Python package can be implemented in your training script.

![train-assess-tune-register](./images/triningjob.png)

### Set the hyperparameters
These are the parameters which will be sent to our training script in order to train the model. Although they are all defined as "hyperparameters" here, they can encompass XGBoost's [Learning Task Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters), [Tree Booster Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster), or any other parameters you'd like to configure for XGBoost.

In [6]:
hyperparameters = {
    "max_depth": "3",
    "eta": "0.2",
    "objective": "binary:logistic",
    "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


### Create and fit the estimator
If you want to explore the breadth of functionailty offered by the SageMaker XGBoost Framework you can read about all the configuration parameters by referencing the inhereting classes. The XGBoost class inherets from the Framework class and Framework inherets from the EstimatorBase class:
* [XGBoost Estimator documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html#sagemaker.xgboost.estimator.XGBoost)
* [Framework documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework)
* [EstimatorBase documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)

In [7]:
xgb_estimator = XGBoost(
    entry_point="code/xgboost_starter_script.py",
    output_path=estimator_output_path,
    code_location=estimator_output_path,
    hyperparameters=hyperparameters,
    role=sagemaker_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    framework_version="1.0-1",
)

In [8]:
if 'training_job_1_name' not in locals():
    
    xgb_estimator.fit(inputs = {'train': train_data_uri})
    training_job_1_name = xgb_estimator.latest_training_job.job_name
    %store training_job_1_name
    
else:
    print(f'Using previous training job: {training_job_1_name}')

2021-11-18 02:10:30 Starting - Starting the training job...
2021-11-18 02:10:54 Starting - Launching requested ML instancesProfilerReport-1637201430: InProgress
......
2021-11-18 02:11:59 Starting - Preparing the instances for training...............
2021-11-18 02:14:15 Downloading - Downloading input data...
2021-11-18 02:14:55 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Created wheel for xgboost-starter-script: filename=xgboo

<a id='model-lineage'></a>

## Model lineage with artifacts and associations

[Overview](#aud-overview)
----
Amazon SageMaker ML Lineage Tracking creates and stores information about the steps of a machine learning (ML) workflow from data preparation to model deployment. With the tracking information you can reproduce the workflow steps, track model and dataset lineage, and establish model governance and audit standards. With SageMaker Lineage Tracking data scientists and model builders can do the following:
* Keep a running history of model discovery experiments.
* Establish model governance by tracking model lineage artifacts for auditing and compliance verification.
* Clone and rerun workflows to experiment with what-if scenarios while developing models.
* Share a workflow that colleagues can reproduce and enhance (for example, while collaborating on solving a business problem).
* Clone and rerun workflows with additional debugging or logging routines, or new input variations for troubleshooting issues in production models.




![train-assess-tune-register](./images/lineage2.png)

In [9]:
from datetime import datetime
from sagemaker.lineage.context import Context
from sagemaker.lineage.action import Action
from sagemaker.lineage.association import Association
from sagemaker.lineage.artifact import Artifact

unique_id = str(int(datetime.now().replace(microsecond=0).timestamp()))

print(f"Unique id is {unique_id}")

Unique id is 1637201755


<a id='register-artifacts'></a>

### Register artifacts

Although the `xgb_estimator` object retains much the data we need to learn about how the model was trained, it is, in fact, an ephermeral object which SageMaker does not persist and cannot be re-instantiated at a later time. Although we lose some of its convieneces once it is gone, we can still get back all the data we need by accessing the training jobs it once created.

In [10]:
training_job_1_info = sagemaker_client.describe_training_job(
    TrainingJobName=training_job_1_name
)
training_job_1_info

{'TrainingJobName': 'sagemaker-xgboost-2021-11-18-02-10-30-067',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:875692608981:training-job/sagemaker-xgboost-2021-11-18-02-10-30-067',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '"0.2"',
  'max_depth': '"3"',
  'num_round': '"100"',
  'objective': '"binary:logistic"',
  'sagemaker_container_log_level': '20',
  'sagemaker_job_name': '"sagemaker-xgboost-2021-11-18-02-10-30-067"',
  'sagemaker_program': '"xgboost_starter_script.py"',
  'sagemaker_region': '"us-east-1"',
  'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/source/sourcedir.tar.gz"'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.

#### Code artifact

In [11]:
# return any existing artifact which match the our training job's code arn
# ====>

# extract the training code uri and check if it's an exisiting artifact
code_s3_uri = training_job_1_info["HyperParameters"]["sagemaker_submit_directory"]
print(f"Code path: {code_s3_uri }")
matching_artifacts = list(Artifact.list(
        source_uri=code_s3_uri, sagemaker_session=sagemaker_session
    )
)

# use existing arifact if it's already been created, otherwise create a new artifact
if matching_artifacts:
    code_artifact = matching_artifacts[0]
    print(f"Using existing artifact: {code_artifact.artifact_arn}")
else:
    code_artifact = Artifact.create(
        artifact_name="TrainingScript",
        source_uri=code_s3_uri,
        artifact_type="Code",
        sagemaker_session=sagemaker_session,
    )
    print(f"Create artifact {code_artifact.artifact_arn}: SUCCESSFUL")

Code path: "s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/source/sourcedir.tar.gz"
Create artifact arn:aws:sagemaker:us-east-1:875692608981:artifact/e0b2728d82c85f32bc13dd35d8411ed5: SUCCESSFUL


#### Training data artifact

In [12]:
training_data_s3_uri = training_job_1_info["InputDataConfig"][0]["DataSource"]["S3DataSource"][
    "S3Uri"
]
print(f"Training Data path: {training_data_s3_uri}")

matching_artifacts = list(Artifact.list(
        source_uri=training_data_s3_uri, sagemaker_session=sagemaker_session
    )
)

if matching_artifacts:
    training_data_artifact = matching_artifacts[0]
    print(f"Using existing artifact: {training_data_artifact.artifact_arn}")
else:
    training_data_artifact = Artifact.create(
        artifact_name="TrainingData",
        source_uri=training_data_s3_uri,
        artifact_type="Dataset",
        sagemaker_session=sagemaker_session,
    )
    print(f"Create artifact {training_data_artifact.artifact_arn}: SUCCESSFUL")

Training Data path: s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data/train/train.csv
Using existing artifact: arn:aws:sagemaker:us-east-1:875692608981:artifact/576a77a77c3ee7805f9d270acdecf07c


#### Model artifact

In [13]:
trained_model_s3_uri = training_job_1_info["ModelArtifacts"]["S3ModelArtifacts"]
print(f"Training Model path: {trained_model_s3_uri}")

matching_artifacts = list(Artifact.list(
        source_uri=trained_model_s3_uri, sagemaker_session=sagemaker_session
    )
)

if matching_artifacts:
    model_artifact = matching_artifacts[0]
    print(f"Using existing artifact: {model_artifact.artifact_arn}")
else:
    model_artifact = Artifact.create(
        artifact_name="TrainedModel",
        source_uri=trained_model_s3_uri,
        artifact_type="Model",
        sagemaker_session=sagemaker_session,
    )
    print(f"Create artifact {model_artifact.artifact_arn}: SUCCESSFUL")

Training Model path: s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/output/model.tar.gz
Using existing artifact: arn:aws:sagemaker:us-east-1:875692608981:artifact/b5c35edf1881a26eccb1b5dbf30eeb68


<a id='Set-artifact-associations'></a>

### Set artifact associations

In [14]:
trial_component = sagemaker_client.describe_trial_component(
    TrialComponentName=training_job_1_name + "-aws-training-job"
)
trial_component_arn = trial_component["TrialComponentArn"]
trial_component_arn

'arn:aws:sagemaker:us-east-1:875692608981:experiment-trial-component/sagemaker-xgboost-2021-11-18-02-10-30-067-aws-training-job'

#### Input artifacts

In [15]:
input_artifacts = [code_artifact, training_data_artifact]

for a in input_artifacts:
    try:
        Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type="ContributedTo",
            sagemaker_session=sagemaker_session,
        )
        print(f"Association with {a.artifact_type}: SUCCEESFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Code: SUCCEESFUL
Association already exists with DataSet


#### Output artifacts

In [16]:
output_artifacts = [model_artifact]

for a in output_artifacts:
    try:
        Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type="Produced",
            sagemaker_session=sagemaker_session,
        )
        print(f"Association with {a.artifact_type}: SUCCESSFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Model: SUCCESSFUL


In [17]:
# create an context
# the name must be unique across all other contexts
context_name = f"fraud-detect-{unique_id}"
%store context_name

ml_workflow_context = Context.create(
    context_name=context_name,
    context_type="AutomobileInsurance",
    source_uri=unique_id,
    # properties services as a method to store metdata on lineage entities in additional to Tags
    properties={"Description": "First Training Job Run"},
)

Stored 'context_name' (str)


In [18]:
# list all the contexts

contexts = Context.list(sort_by="CreationTime", sort_order="Descending")

for ctx in contexts:
    print(ctx.context_name)

fraud-detect-1637201755
xgboost-model-pipeline-0120-1637150367-aws-endpoint
fraud-detect-demo-xgboost-post-smote-endpoint-1637147316-aws-endpoint


In [19]:
# create an example action and associate it with the context

model_build_action = Action.create(
    action_name=f"model-train-{unique_id}",
    action_type="ModelBuild",
    source_uri=trial_component_arn,
    properties={"Example": "Metadata"},
)

In [20]:
# Association Type can be Produced|DerivedFrom|AssociatedWith|ContributedTo
context_action_association = Association.create(
    source_arn=ml_workflow_context.context_arn,
    destination_arn=trial_component_arn,
    association_type="AssociatedWith",
)

## Tracking Lineage

In [21]:
#Tracking Dataset Source Lineage
train_data_location_artifact_arn = sagemaker_client.list_associations(
    DestinationArn=trial_component_arn, SourceType='DataSet')['AssociationSummaries'][0]['SourceArn']
# print(train_data_location_artifact_arn)

train_data_location = sagemaker_client.describe_artifact(ArtifactArn=train_data_location_artifact_arn)['Source']['SourceUri']
print(train_data_location)

s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data/train/train.csv


In [22]:
#Tracking Dataset Model Lineage
train_data_location_artifact_arn = sagemaker_client.list_associations(
    DestinationArn=trial_component_arn, SourceType='Model')['AssociationSummaries'][0]['SourceArn']
# print(train_data_location_artifact_arn)

train_data_location = sagemaker_client.describe_artifact(ArtifactArn=train_data_location_artifact_arn)['Source']['SourceUri']
print(train_data_location)

s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/output/model.tar.gz


In [23]:
#Tracking Dataset Code Lineage
train_data_location_artifact_arn = sagemaker_client.list_associations(
    DestinationArn=trial_component_arn, SourceType='Code')['AssociationSummaries'][0]['SourceArn']
# print(train_data_location_artifact_arn)

train_data_location = sagemaker_client.describe_artifact(ArtifactArn=train_data_location_artifact_arn)['Source']['SourceUri']
print(train_data_location)

"s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/source/sourcedir.tar.gz"


<a id='check-bias'></a>

## Evaluate model for bias with Clarify

[overview](#aud-overview)
----
Amazon SageMaker Clarify helps improve your machine learning (ML) models by detecting potential bias and helping explain the predictions that models make. It helps you identify various types of bias in pretraining data and in posttraining that can emerge during model training or when the model is in production. SageMaker Clarify helps explain how these models make predictions using a feature attribution approach. It also monitors inferences models make in production for bias or feature attribution drift. The fairness and explainability functionality provided by SageMaker Clarify provides components that help AWS customers build less biased and more understandable machine learning models. It also provides tools to help you generate model governance reports which you can use to inform risk and compliance teams, and external regulators. 

You can reference the [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html) for more information about SageMaker Clarify.


![train-assess-tune-register](./images/clarify.png)

### Create model from estimator

In [24]:
model_1_name = f"{prefix}-xgboost-pre-smote"
%store model_1_name
model_matches = sagemaker_client.list_models(NameContains=model_1_name)["Models"]

if not model_matches:

    model_1 = sagemaker_session.create_model_from_job(
        name=model_1_name,
        training_job_name=training_job_1_info["TrainingJobName"],
        role=sagemaker_role,
        image_uri=training_job_1_info["AlgorithmSpecification"]["TrainingImage"],
    )
else:

    print(f"Model {model_1_name} already exists.")

Stored 'model_1_name' (str)


<a id='bias-v1'></a>

### Check for data set bias and model bias

With SageMaker, we can check for pre-training and post-training bias. Pre-training metrics show pre-existing bias in that data, while post-training metrics show bias in the predictions from the model. Using the SageMaker SDK, we can specify which groups we want to check bias across and which metrics we'd like to show. 

To run the full Clarify job, you must un-comment the code in the cell below. Running the job will take ~15 minutes. If you wish to save time, you can view the results in the next cell after which loads a pre-generated output if no bias job was run.

In [25]:
train_cols = wr.s3.read_csv(training_data_s3_uri).columns.to_list()

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=sagemaker_role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_uri,
    s3_output_path=bias_report_1_output_path,
    label="fraud",
    headers=train_cols,
    dataset_type="text/csv",
)

model_config = sagemaker.clarify.ModelConfig(
    model_name=model_1_name,
    instance_type=train_instance_type,
    instance_count=1,
    accept_type="text/csv",
)

predictions_config = sagemaker.clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name="customer_gender_female",
    facet_values_or_threshold=[1],
)

# un-comment the code below to run the whole job

if 'clarify_bias_job_1_name' not in locals():

    clarify_processor.run_bias(
        data_config=bias_data_config,
        bias_config=bias_config,
        model_config=model_config,
        model_predicted_label_config=predictions_config,
        pre_training_methods='all',
        post_training_methods='all')

    clarify_bias_job_1_name = clarify_processor.latest_job.name
    %store clarify_bias_job_1_name

else:
    print(f'Clarify job {clarify_bias_job_name} has already run successfully.')


Job Name:  Clarify-Bias-2021-11-18-02-16-43-192
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/data/train/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/clarify-output/bias_1/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/clarify-output/bias_1', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
................................

Results will be stored in `/opt/ml/processing/output/report.pdf`
Training to achieve over 90 percent classification accuracy, may be easily possible on an imbalanced classification problem.

Thus, expectations developed regarding classification accuracy that are in reality contingent on balanced class distributions will lead to wrong, misleading assumptions and conclusions : misleading the data scientist and viewers into believing that a model has extremely performance when , actually, it does not.

### View results of Clarify job (shortcut)
Running Clarify on your dataset or model can take ~15 minutes. If you don't have time to run the job, you can view the pre-generated results included with this demo. Otherwise, you can run the job by un-commenting the code in the cell above.

In [26]:
if "clarify_bias_job_1_name" in locals():
    s3_client.download_file(
        Bucket=bucket,
        Key=f"{prefix}/clarify-output/bias_1/analysis.json",
        Filename="clarify_output/bias_1/analysis.json",
    )
    print(f"Downloaded analysis from previous Clarify job: {clarify_bias_job_1_name}")
else:
    print(f"Loading pre-generated analysis file...")

with open("clarify_output/bias_1/analysis.json", "r") as f:
    bias_analysis = json.load(f)

results = bias_analysis["pre_training_bias_metrics"]["facets"]["customer_gender_female"][0]["metrics"][1]
print(json.dumps(results, indent=4))

Downloaded analysis from previous Clarify job: Clarify-Bias-2021-11-18-02-16-43-192
{
    "name": "CI",
    "description": "Class Imbalance (CI)",
    "value": 0.114
}


In this example dataset, the data is biased against females with only 38.9% of the data samples from female customers. We will address this in the next notebook where we show how we mitigate this class imbalance bias. Although we are only addressing Class Imbalance as an exemplar of bias statistics, you can also take into consideration many other factors of bias. For more detail, see : [Fairness Measures for Machine Learning in Finance](https://pages.awscloud.com/rs/112-TZM-766/images/Fairness.Measures.for.Machine.Learning.in.Finance.pdf)

for a more detailed example look at [this](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) github example.

For more detailed resulst let's look at the generated report, that can be found here: `s3://{bucket}/e2e-fraud-detect/clarify/bias-2/report.pdf`

In [27]:
# uncomment to copy report and view
!aws s3 cp s3://{bucket}/{prefix}/clarify-output/bias_1/report.pdf ./clarify_output

download: s3://sagemaker-us-east-1-875692608981/fraud-detect-demo/clarify-output/bias_1/report.pdf to clarify_output/report.pdf


<a id='model-registry'></a>

## Deposit Model and Lineage in SageMaker Model Registry

[overview](#aud-overview)
----
Once a useful model has been trained and its artifacts properly associated, the next step is to save the model in a registry for future reference and possible deployment.



![train-assess-tune-register](./images/modelregistry.png)

### Create Model Package Group
A Model Package Groups holds multiple versions or iterations of a model. Though it is not required to create them for every model in the registry, they help organize various models which all have the same purpose and provide automatic versioning.

In [28]:
if 'mpg_name' not in locals():
    mpg_name = prefix
    %store mpg_name
    print(f'Model Package Group name: {mpg_name}')

Stored 'mpg_name' (str)
Model Package Group name: fraud-detect-demo


In [29]:
mpg_input_dict = {
    "ModelPackageGroupName": mpg_name,
    "ModelPackageGroupDescription": "Insurance claim fraud detection",
}

In [30]:
matching_mpg = sagemaker_client.list_model_package_groups(NameContains=mpg_name)['ModelPackageGroupSummaryList']

if matching_mpg:
    print(f'Using existing Model Package Group: {mpg_name}')
else:
    mpg_response = sagemaker_client.create_model_package_group(**mpg_input_dict)
    print(f'Create Model Package Group {mpg_name}: SUCCESSFUL')
    %store mpg_name

Create Model Package Group fraud-detect-demo: SUCCESSFUL
Stored 'mpg_name' (str)


### Create Model Package for trained model

#### Create and upload a metrics report

In [31]:
model_metrics_report = {"binary_classification_metrics": {}}
for metric in training_job_1_info["FinalMetricDataList"]:
    stat = {metric["MetricName"]: {"value": metric["Value"], "standard_deviation": "NaN"}}
    model_metrics_report["binary_classification_metrics"].update(stat)
    print(metric)

with open("config/training_metrics.json", "w") as f:
    json.dump(model_metrics_report, f)

metrics_s3_key = (
    f"{prefix}/training_jobs/{training_job_1_info['TrainingJobName']}/training_metrics.json"
)
s3_client.upload_file(Filename="config/training_metrics.json", Bucket=bucket, Key=metrics_s3_key)

{'MetricName': 'validation:auc', 'Value': 0.8221806287765503, 'Timestamp': datetime.datetime(2021, 11, 18, 2, 15, 11, tzinfo=tzlocal())}
{'MetricName': 'train:auc', 'Value': 0.8979467749595642, 'Timestamp': datetime.datetime(2021, 11, 18, 2, 15, 11, tzinfo=tzlocal())}


#### Define the inference spec

In [32]:
mp_inference_spec = InferenceSpecification().get_inference_specification_dict(
    ecr_image=training_job_1_info["AlgorithmSpecification"]["TrainingImage"],
    supports_gpu=False,
    supported_content_types=["text/csv"],
    supported_mime_types=["text/csv"],
)

mp_inference_spec["InferenceSpecification"]["Containers"][0]["ModelDataUrl"] = training_job_1_info["ModelArtifacts"]["S3ModelArtifacts"]
print(mp_inference_spec)

{'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/output/model.tar.gz'}], 'SupportedTransformInstanceTypes': ['ml.m4.xlarge', 'ml.m4.2xlarge', 'ml.m4.4xlarge', 'ml.m4.10xlarge', 'ml.m4.16xlarge', 'ml.m5.large', 'ml.m5.xlarge', 'ml.m5.2xlarge', 'ml.m5.4xlarge', 'ml.m5.12xlarge', 'ml.m5.24xlarge', 'ml.c4.xlarge', 'ml.c4.2xlarge', 'ml.c4.4xlarge', 'ml.c4.8xlarge', 'ml.c5.xlarge', 'ml.c5.2xlarge', 'ml.c5.4xlarge', 'ml.c5.9xlarge', 'ml.c5.18xlarge'], 'SupportedRealtimeInferenceInstanceTypes': ['ml.m4.xlarge', 'ml.m4.2xlarge', 'ml.m4.4xlarge', 'ml.m4.10xlarge', 'ml.m4.16xlarge', 'ml.m5.large', 'ml.m5.xlarge', 'ml.m5.2xlarge', 'ml.m5.4xlarge', 'ml.m5.12xlarge', 'ml.m5.24xlarge', 'ml.c4.xlarge', 'ml.c4.2xlarge', 'ml.c4.4xlarge', 'ml.c4.8xlarge', 'ml.c5.xlarge', 'ml.c5.2xlarge', 'ml.c5.

#### Define model metrics
Metrics other than model quality and bias can be defined. See the Boto3 documentation for [creating a model package](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model_package).

In [33]:
model_metrics = {
    "ModelQuality": {
        "Statistics": {
            "ContentType": "application/json",
            "S3Uri": f"s3://{bucket}/{metrics_s3_key}",
        }
    },
    "Bias": {
        "Report": {
            "ContentType": "application/json",
            "S3Uri": f"{bias_report_1_output_path}/analysis.json",
        }
    },
}
print(model_metrics)

{'ModelQuality': {'Statistics': {'ContentType': 'application/json', 'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/training_jobs/sagemaker-xgboost-2021-11-18-02-10-30-067/training_metrics.json'}}, 'Bias': {'Report': {'ContentType': 'application/json', 'S3Uri': 's3://sagemaker-us-east-1-875692608981/fraud-detect-demo/clarify-output/bias_1/analysis.json'}}}


In [34]:
mp_input_dict = {
    "ModelPackageGroupName": mpg_name,
    "ModelPackageDescription": "XGBoost classifier to detect insurance fraud.",
    "ModelApprovalStatus": "PendingManualApproval",
    "ModelMetrics": model_metrics,
}

mp_input_dict.update(mp_inference_spec)
mp1_response = sagemaker_client.create_model_package(**mp_input_dict)
print(mp1_response)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:875692608981:model-package/fraud-detect-demo/1', 'ResponseMetadata': {'RequestId': '00cae0a6-3e1f-4d44-832f-39cfe765dace', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '00cae0a6-3e1f-4d44-832f-39cfe765dace', 'content-type': 'application/x-amz-json-1.1', 'content-length': '96', 'date': 'Thu, 18 Nov 2021 02:29:11 GMT'}, 'RetryAttempts': 0}}


### Wait until model package is completed

In [35]:
mp_info = sagemaker_client.describe_model_package(
    ModelPackageName=mp1_response["ModelPackageArn"]
)
mp_status = mp_info["ModelPackageStatus"]

while mp_status not in ["Completed", "Failed"]:
    time.sleep(5)
    mp_info = sagemaker_client.describe_model_package(
        ModelPackageName=mp1_response["ModelPackageArn"]
    )
    mp_status = mp_info["ModelPackageStatus"]
    print(f"model package status: {mp_status}")
print(f"model package status: {mp_status}")

model package status: Completed


### View model package in registry

In [36]:
sagemaker_client.list_model_packages(ModelPackageGroupName=mpg_name)["ModelPackageSummaryList"]

[{'ModelPackageGroupName': 'fraud-detect-demo',
  'ModelPackageVersion': 1,
  'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:875692608981:model-package/fraud-detect-demo/1',
  'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
  'CreationTime': datetime.datetime(2021, 11, 18, 2, 29, 11, 779000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'PendingManualApproval'}]

----

### Next Notebook: [Mitigate Bias, Train New Model, Store in Registry](./05-Mitigate-Bias-Train-Model2-Registry.ipynb)

To handle the imbalance, in the next notebook, we over-sample (i.e. upsample) the minority class using [SMOTE (Synthetic Minority Over-sampling Technique)](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html).